La voiture : étude d'un fait social

Par Alexandre Pakin et Gilles Massoulié

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Introduction

L’usage de la voiture constitue un objet central d’analyse en sociologie, tant il structure les mobilités quotidiennes, les trajectoires sociales et les rapports aux territoires en France. Loin d’être un simple outil de déplacement, l’automobile peut être appréhendée comme un fait social, révélateur d’inégalités de classe, de genre, d’âge et de territoire.

Cette étude a été motivée par une [Vidéo de vulgarisation sociologique sur l’automobile](https://www.youtube.com/watch?v=tR6aujaUh_g)
 proposant une synthèse critique de plusieurs travaux récents sur l’automobile et ses usages sociaux. En mettant en évidence la dimension profondément inégalitaire de la conduite et de la possession d’un véhicule, cette vidéo invite à revenir aux données empiriques afin de reproduire et discuter certaines conclusions à partir de sources ouvertes. Le présent projet s’inscrit ainsi dans une démarche de réanalyse quantitative, visant à confronter des hypothèses sociologiques établies aux données disponibles.

La littérature récente permet de dégager plusieurs axes majeurs. D’une part, des travaux généraux de sociologie de l’automobile, comme [Sociologie de l’automobile](https://www.cairn.info/sociologie-de-l-automobile--9782707196433.htm) de Jean-Pierre Orfeuil et Frédéric Héran (2022) ou [Idées reçues sur l’automobile](https://www.cairn.info/idees-recues-sur-l-automobile--9782715405666.htm) de Frédéric Héran (2021), proposent un cadre théorique permettant de déconstruire les représentations dominantes associées à la voiture. D’autre part, plusieurs études mettent en évidence des usages différenciés selon les caractéristiques sociales des individus. Les dimensions d’âge et de genre sont notamment abordées dans des travaux portant sur la jeunesse automobile ou sur les inégalités et comportements genrés au volant, tandis que d’autres recherches analysent l’automobile comme un marqueur de position sociale et un révélateur des inégalités de classe, notamment à travers les politiques de mobilité et de sécurité routière.

Dans le prolongement de ces travaux, ce projet vise à analyser empiriquement l’usage de la voiture en France à partir de données issues de trois API publiques (data.gouv.fr, INSEE et transport.data.gouv.fr). L’objectif est d’identifier des régularités statistiques et des inégalités sociales liées à l’automobile, tout en illustrant l’apport des outils de programmation en Python pour l’analyse quantitative de phénomènes sociaux.

Sommaire (à faire)

Installation

In [12]:
!pip install -r requirements.txt

# modules 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
import re


# Fonctions
from urllib import request
from modules import module_BAAC
from modules import module_transport
from modules import graph_baac

Préparation des données

1. Adresses

Toutes les données utilisées dans ce projet sont récupérées directement en ligne, principalement via des interfaces de programmation (API) publiques. Ces API permettent un accès automatisé et reproductible aux données, ce qui s’inscrit pleinement dans une démarche de science des données appliquée aux sciences sociales.

In [13]:
# API Bases de données annuelles des accidents corporels de la circulation routière – 2005 à 2024
BAAC_url = "https://www.data.gouv.fr/api/1/datasets/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/"

# API INSEE (statistiques socio-économiques)
insee_url = "https://api.insee.fr/series/BDM/V1/data"

# API transport.data.gouv.fr (mobilité et trafic)
transport_url = "https://transport.data.gouv.fr/api/datasets"

2. Liste des accidents

(à compléter, chaque ligne est un accident)

In [14]:
BAAC_resources = module_BAAC.get_resources(BAAC_url)
pd.set_option("display.max_rows", None)
pd.options.display.max_colwidth = 300
BAAC_resources[["description", "url"]].head(10)

,description,url
0,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2024.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100210/2024.csv
1,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2023.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142813/2023.csv
2,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv
3,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv
4,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv
5,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv
6,Liste des véhicules immatriculés (hors vélos et engins de déplacement personnel) impliqués dans les accidents corporels issus des BAAC (Bulletin d’Analyse des Accidents Corporels de la circulation) relevés par les forces de l’ordre en 2022.\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-142722/2022.csv
7,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv
8,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv
9,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv


On a sélectionné les catégories suivantes (à rédiger), toutes les tables sauf la première qui est une liste des véhicules immatriculés dans les accidents ont en lien la variable num_acc, donc on les fusionne pour obtenir le dataframe BAAC_table.
Il faudra qd mm utiliser le premier dataframe car il est sûrement intéressant.

In [ ]:
selected_BAAC_table = module_BAAC.select_baac_tables(BAAC_resources)
selected_BAAC_table.head(10)

,description,url,table,year
0,Table des caractéristiques des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115900/caract-2024.csv,caracteristiques,2024
1,Table des lieux des accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115812/lieux-2024.csv,lieux,2024
2,Table des véhicules impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251107-100240/vehicules-2024.csv,vehicules,2024
3,Table des usagers impliqués dans les accidents corporels de la circulation en 2024 en France (France métropolitaine + DOM + COM +NC).\n,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2024/20251021-115506/usagers-2024.csv,usagers,2024
4,Table des usagers impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153328/usagers-2023.csv,usagers,2023
5,Table des véhicules impliqués dans les accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153253/vehicules-2023.csv,vehicules,2023
6,Table des lieux des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241023-153219/lieux-2023.csv,lieux,2023
7,Table des caractéristiques des accidents corporels de la circulation en 2023 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2023/20241028-103125/caract-2023.csv,caracteristiques,2023
8,Table des usagers impliqués dans les accidents corporels de la circulation en 2022 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094229/usagers-2022.csv,usagers,2022
9,Table des véhicules impliqués dans les accidents corporels de la circulation en 2022 en France (France métropolitaine + DOM + COM +NC).,https://static.data.gouv.fr/resources/bases-de-donnees-annuelles-des-accidents-corporels-de-la-circulation-routiere-annees-de-2005-a-2021/20231005-094147/vehicules-2022.csv,vehicules,2022


ça marche mais ça prends 3min34s à tourner

In [16]:
BAAC_table = module_BAAC.build_baac_dataframe(selected_BAAC_table)
BAAC_table.head(10)

,Num_Acc,place,catu,grav,sexe,trajet,secu,locp,actp,etatp,...,dep,year,id_vehicule_x,secu1,secu2,secu3,id_vehicule_y,motor,vma,id_usager
0,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,200500000001,1,1,4,1,1,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,200500000001,1,1,3,2,3,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,200500000001,2,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,200500000001,2,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,200500000001,4,2,1,1,0,31,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,200500000001,4,2,1,1,0,31,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,200500000001,5,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,200500000001,5,2,1,1,0,11,0,0,0,...,590,2005,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Heatmap en fonction de l'heure et du jour de la semaine

In [17]:
BAAC_table['hour'] = BAAC_table['hrmn'].str.split(':').str[0].astype(int)
BAAC_table.columns

BAAC_table.to_csv('table_test.csv')

ValueError: cannot convert float NaN to integer

In [ ]:
BAAC_table[['lat','long']].head()

,lat,long
0,"47,56277000","6,75832000"
1,"47,02109000","4,83755000"
2,"47,02109000","4,83755000"
3,"44,90238400","2,49641800"
4,"49,19166000","-0,39851000"


blabla 2e dataframe

In [27]:
transport_resources = module_transport.get_resources(transport_url)["resources"]
transport_urls = module_transport.extract_resources_urls(transport_resources)
transport_urls

,url,title
0,https://transport.data.gouv.fr/resources/81372/download,bnlc.csv
1,https://transport.data.gouv.fr/resources/83036/download,Jeux de données retirés temporairement
2,https://transport.data.gouv.fr/resources/18641/download,BNLC (avant janvier 2024)
3,https://transport.data.gouv.fr/resources/81795/download,Jeux de données composant la BNLC
4,https://transport.data.gouv.fr/resources/79489/download,aire-de-covoiturage-bretagne-supplementaires.shp
5,https://transport.data.gouv.fr/resources/78899/download,Base Nationale de stationnement hors voirie (non exhaustive)
6,https://transport.data.gouv.fr/resources/79896/download,basep-r-05012023-3-.csv
7,https://transport.data.gouv.fr/resources/79568/download,voies.geojson
8,https://transport.data.gouv.fr/resources/79760/download,Identifiants des ZFE
9,https://transport.data.gouv.fr/resources/79567/download,aires.geojson
